In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import numpy as np
import pandas as pd
import math
import json
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
# from sklearn.externals import joblib
import scipy.sparse
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
import warnings; warnings.simplefilter('ignore')
import json
%matplotlib inline

In [ ]:
data = []
with open('Electronics_5.json') as f:
    for i, line in enumerate(f):
        if i == 5000: # stop after reading 5000 lines
            break
        data.append(json.loads(line))

df = pd.DataFrame(data)

In [ ]:
specified_columns = ['reviewerID', 'asin', 'overall']
df = df[specified_columns]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   reviewerID  5000 non-null   object 
 1   asin        5000 non-null   object 
 2   overall     5000 non-null   float64
dtypes: float64(1), object(2)
memory usage: 117.3+ KB


In [ ]:
df.head()

,reviewerID,asin,overall
0,AO94DHGC771SJ,0528881469,5.0
1,AMO214LNFCEI4,0528881469,1.0
2,A3N7T0DY83Y4IG,0528881469,3.0
3,A1H8PY3QHMQQA0,0528881469,2.0
4,A24EV6RXELQZ63,0528881469,1.0


In [ ]:
#Summary statistics of rating variable
df['overall'].describe().transpose()

count    5000.000000
mean        4.230000
std         1.170544
min         1.000000
25%         4.000000
50%         5.000000
75%         5.000000
max         5.000000
Name: overall, dtype: float64

In [ ]:
#Find the minimum and maximum ratings
print('Minimum rating is: %d' %(df.overall.min()))
print('Maximum rating is: %d' %(df.overall.max()))

Minimum rating is: 1
Maximum rating is: 5


In [ ]:
#Check for missing values
print('Number of missing values across columns: \n',df.isnull().sum())

Number of missing values across columns: 
 reviewerID    0
asin          0
overall       0
dtype: int64


In [ ]:
# Number of unique user id  in the data
print('Number of unique users in data = ', df['reviewerID'].nunique())
# Number of unique product id  in the data
print('Number of unique product in data = ', df['asin'].nunique())

Number of unique users in data =  4627
Number of unique product in data =  199


In [ ]:
#Check the top 10 users based on ratings
most_rated=df.groupby('reviewerID').size().sort_values(ascending=False)[:10]
print('Top 10 users based on ratings: \n',most_rated)

Top 10 users based on ratings: 
 reviewerID
A3PLX6PTM2ERKL    6
A3A15L96IYUO6V    6
A6ZPLVAUQ6695     6
A3LDPF5FMB782Z    5
ASWYSSKL9LL9X     5
A3MEIR72XKQY88    5
AN30G4IKL1BMZ     4
A204ETWOV23HO4    4
A2BLITJITO97N5    4
A11ED8O95W2103    4
dtype: int64


In [ ]:
counts=df.reviewerID.value_counts()
df_final=df[df.reviewerID.isin(counts[counts>=5].index)]
print('Number of users who have rated 5 or more items =', len(df_final))
print('Number of unique users in the final data = ', df_final['reviewerID'].nunique())
print('Number of unique products in the final data = ', df_final['reviewerID'].nunique())

Number of users who have rated 5 or more items = 33
Number of unique users in the final data =  6
Number of unique products in the final data =  6


In [ ]:
#constructing the pivot table
final_ratings_matrix = df_final.pivot(index = 'reviewerID', columns ='asin', values = 'overall').fillna(0)
final_ratings_matrix.head()

asin,1400501466,1400501520,1400501776,140053271X,1400698987,B00000IGBF,B00000J1TX,B00000J3II,B00000J3Q1,B00000J434,...,B00000JDK0,B00000JI2C,B00000JI2D,B00000JI38,B00000JSGF,B00000JYLO,B00001MXYW,B00001P4XA,B00001P4ZH,B00001RMCY
reviewerID,,,,,,,,,,,,,,,,,,,,,
A3A15L96IYUO6V,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,4.0,0.0,0.0
A3LDPF5FMB782Z,5.0,5.0,5.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A3MEIR72XKQY88,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,5.0,0.0,5.0,2.0,0.0
A3PLX6PTM2ERKL,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0,...,5.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A6ZPLVAUQ6695,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,5.0,5.0,0.0,0.0,0.0,0.0,5.0


In [ ]:
print('Shape of final_ratings_matrix: ', final_ratings_matrix.shape)

Shape of final_ratings_matrix:  (6, 28)


In [ ]:
#Calucating the density of the rating marix
given_num_of_ratings = np.count_nonzero(final_ratings_matrix)
print('given_num_of_ratings = ', given_num_of_ratings)

possible_num_of_ratings = final_ratings_matrix.shape[0] * final_ratings_matrix.shape[1]
print('possible_num_of_ratings = ', possible_num_of_ratings)

density = (given_num_of_ratings/possible_num_of_ratings)
density *= 100
print ('density: {:4.2f}%'.format(density))

given_num_of_ratings =  33
possible_num_of_ratings =  168
density: 19.64%


In [ ]:
#Split the data randomnly into train and test datasets into 80:20 ratio
train_data, test_data = train_test_split(df_final, test_size = 0.2, random_state=0)
train_data.head()

,reviewerID,asin,overall
2349,A3MEIR72XKQY88,B00000JBIA,1.0
2122,ASWYSSKL9LL9X,B00000J434,5.0
4549,A6ZPLVAUQ6695,B00001RMCY,5.0
3617,A3A15L96IYUO6V,B00001P4XA,4.0
2803,A3PLX6PTM2ERKL,B00000JI2D,5.0


In [ ]:
print('Shape of training data: ',train_data.shape)
print('Shape of testing data: ',test_data.shape)

Shape of training data:  (26, 3)
Shape of testing data:  (7, 3)


In [ ]:
#Count of reviewer_id for each unique product as recommendation score
train_data_grouped = train_data.groupby('asin').agg({'reviewerID': 'count'}).reset_index()
train_data_grouped.rename(columns = {'reviewerID': 'score'},inplace=True)
train_data_grouped.head(40)

,asin,score
0,1400501466,1
1,1400501520,1
2,140053271X,1
3,1400698987,1
4,B00000IGBF,1
5,B00000J1TX,1
6,B00000J3II,1
7,B00000J3Q1,2
8,B00000J434,1
9,B00000J47F,1


In [ ]:
#Sort the products on recommendation score
train_data_sort = train_data_grouped.sort_values(['score', 'asin'], ascending = [0,1])

#Generate a recommendation rank based upon score
train_data_sort['rank'] = train_data_sort['score'].rank(ascending=0, method='first')

#Get the top 5 recommendations
popularity_recommendations = train_data_sort.head(5)
popularity_recommendations

,asin,score,rank
7,B00000J3Q1,2,1.0
17,B00000JI2D,2,2.0
20,B00001P4XA,2,3.0
0,1400501466,1,4.0
1,1400501520,1,5.0


In [ ]:
# Use popularity based recommender model to make predictions
def recommend(reviewerID):
    user_recommendations = popularity_recommendations

    #Add user_id column for which the recommendations are being generated
    user_recommendations['reviewerID'] = reviewerID

    #Bring user_id column to the front
    cols = user_recommendations.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    user_recommendations = user_recommendations[cols]

    return user_recommendations

In [ ]:
find_recom = [10,100,150]
for i in find_recom:
    print("The list of recommendations for the userId: %d\n" %(i))
    print(recommend(i))
    print("\n")

The list of recommendations for the userId: 10

    reviewerID        asin  score  rank
7           10  B00000J3Q1      2   1.0
17          10  B00000JI2D      2   2.0
20          10  B00001P4XA      2   3.0
0           10  1400501466      1   4.0
1           10  1400501520      1   5.0


The list of recommendations for the userId: 100

    reviewerID        asin  score  rank
7          100  B00000J3Q1      2   1.0
17         100  B00000JI2D      2   2.0
20         100  B00001P4XA      2   3.0
0          100  1400501466      1   4.0
1          100  1400501520      1   5.0


The list of recommendations for the userId: 150

    reviewerID        asin  score  rank
7          150  B00000J3Q1      2   1.0
17         150  B00000JI2D      2   2.0
20         150  B00001P4XA      2   3.0
0          150  1400501466      1   4.0
1          150  1400501520      1   5.0




In [ ]:
df_CF = pd.concat([train_data, test_data]).reset_index()
df_CF.head()

,index,reviewerID,asin,overall
0,2349,A3MEIR72XKQY88,B00000JBIA,1.0
1,2122,ASWYSSKL9LL9X,B00000J434,5.0
2,4549,A6ZPLVAUQ6695,B00001RMCY,5.0
3,3617,A3A15L96IYUO6V,B00001P4XA,4.0
4,2803,A3PLX6PTM2ERKL,B00000JI2D,5.0


In [ ]:
# Matrix with row per 'user' and column per 'item'
pivot_df = df_CF.pivot(index = 'reviewerID', columns ='asin', values = 'overall').fillna(0)
pivot_df.head()

asin,1400501466,1400501520,1400501776,140053271X,1400698987,B00000IGBF,B00000J1TX,B00000J3II,B00000J3Q1,B00000J434,...,B00000JDK0,B00000JI2C,B00000JI2D,B00000JI38,B00000JSGF,B00000JYLO,B00001MXYW,B00001P4XA,B00001P4ZH,B00001RMCY
reviewerID,,,,,,,,,,,,,,,,,,,,,
A3A15L96IYUO6V,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,4.0,0.0,0.0
A3LDPF5FMB782Z,5.0,5.0,5.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A3MEIR72XKQY88,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,5.0,0.0,5.0,2.0,0.0
A3PLX6PTM2ERKL,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0,...,5.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A6ZPLVAUQ6695,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,5.0,5.0,0.0,0.0,0.0,0.0,5.0


In [ ]:
print('Shape of the pivot table: ', pivot_df.shape)

Shape of the pivot table:  (6, 28)


In [ ]:
#define user index from 0 to 10
pivot_df['user_index'] = np.arange(0, pivot_df.shape[0], 1)
pivot_df.head()

asin,1400501466,1400501520,1400501776,140053271X,1400698987,B00000IGBF,B00000J1TX,B00000J3II,B00000J3Q1,B00000J434,...,B00000JI2C,B00000JI2D,B00000JI38,B00000JSGF,B00000JYLO,B00001MXYW,B00001P4XA,B00001P4ZH,B00001RMCY,user_index
reviewerID,,,,,,,,,,,,,,,,,,,,,
A3A15L96IYUO6V,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,5.0,4.0,0.0,0.0,0
A3LDPF5FMB782Z,5.0,5.0,5.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
A3MEIR72XKQY88,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.0,5.0,0.0,5.0,2.0,0.0,2
A3PLX6PTM2ERKL,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0,...,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
A6ZPLVAUQ6695,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,5.0,5.0,0.0,0.0,0.0,0.0,5.0,4


In [ ]:
pivot_df.set_index(['user_index'], inplace=True)
# Actual ratings given by users
pivot_df.head()

asin,1400501466,1400501520,1400501776,140053271X,1400698987,B00000IGBF,B00000J1TX,B00000J3II,B00000J3Q1,B00000J434,...,B00000JDK0,B00000JI2C,B00000JI2D,B00000JI38,B00000JSGF,B00000JYLO,B00001MXYW,B00001P4XA,B00001P4ZH,B00001RMCY
user_index,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,4.0,0.0,0.0
1,5.0,5.0,5.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,5.0,0.0,5.0,2.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0,...,5.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,5.0,5.0,0.0,0.0,0.0,0.0,5.0


In [ ]:
# Singular Value Decomposition

from scipy.sparse import csr_matrix

# Convert DataFrame to sparse matrix
sparse_df = csr_matrix(pivot_df.values)

# Singular Value Decomposition
U, sigma, Vt = svds(sparse_df, k=5)

# U, sigma, Vt = svds(pivot_df, k = 10)

In [ ]:
print('Left singular matrix: \n',U)

Left singular matrix: 
 [[-0.00213739 -0.06099803  0.17524024 -0.20964183 -0.05900029 -0.04867853
  -0.1481366  -0.28630312 -0.11256891 -0.08055806]
 [-0.04850921  0.02451831 -0.07412419  0.16713107 -0.14903745  0.0330769
   0.0171048   0.07969043 -0.2555438  -0.0522743 ]
 [-0.09645437  0.05400296  0.13075157 -0.15870989  0.35140861 -0.52379415
  -0.06366031  0.49068921 -0.51122602 -0.09017635]
 [-0.00492741  0.28233749 -0.03114143 -0.20375443  0.02217459 -0.23363782
   0.34286826 -0.60315455 -0.23812485 -0.13864156]
 [-0.04749623 -0.08709264  0.28122106  0.22318256  0.6392849   0.46363402
  -0.23465734 -0.22689736 -0.13624173 -0.07984947]
 [-0.04759307 -0.05032145 -0.08797919 -0.0940633   0.00420016  0.00391163
  -0.06592998  0.15262038  0.32592066 -0.90293404]
 [ 0.52731819  0.36265232  0.08499641 -0.13131402  0.08372151 -0.00542589
  -0.29750386 -0.0819268  -0.07781535 -0.10423462]
 [-0.02045625 -0.01984738 -0.03135976 -0.02899908  0.00124691  0.0010503
  -0.01620562  0.02612961  0.

In [ ]:
print('Sigma: \n',sigma)

Sigma: 
 [ 8.74185328 10.34408043 11.2821124  11.90582563 12.66269507]


In [ ]:
# Construct diagonal array in SVD
sigma = np.diag(sigma)
print('Diagonal matrix: \n',sigma)

Diagonal matrix: 
 [[ 8.74185328  0.          0.          0.          0.        ]
 [ 0.         10.34408043  0.          0.          0.        ]
 [ 0.          0.         11.2821124   0.          0.        ]
 [ 0.          0.          0.         11.90582563  0.        ]
 [ 0.          0.          0.          0.         12.66269507]]


In [ ]:
print('Right singular matrix: \n',Vt)

Right singular matrix: 
 [[-2.22044605e-16 -1.90501314e-16 -1.90501314e-16 -1.52401051e-16
  -1.52401051e-16  4.83415869e-01 -2.24566054e-01  6.88126029e-02
   2.74488407e-01  3.03763025e-01 -6.73465730e-02  4.83415869e-01
  -2.24566054e-01  6.88126029e-02 -3.20529112e-02  5.50500823e-02
   5.50500823e-02 -1.12244288e-01 -2.24566054e-01 -2.24566054e-01
  -3.11997070e-02 -1.12244288e-01 -2.40455933e-01 -1.60264556e-01
   6.88126029e-02 -1.05214474e-01 -6.41058224e-02 -1.12244288e-01]
 [ 4.83368245e-01  4.83368245e-01  4.83368245e-01  3.86694596e-01
   3.86694596e-01  1.60499855e-16 -3.59080961e-16 -3.35794098e-16
   4.96240070e-16 -1.37497845e-16  2.03024217e-16  1.60499855e-16
  -3.59080961e-16 -3.35794098e-16 -3.79411970e-17 -2.67155175e-16
  -2.73203406e-16  3.26699062e-16 -3.58804326e-16 -3.58804326e-16
  -2.95063916e-16  3.26699062e-16  1.77884492e-16 -1.94691829e-16
  -3.44973705e-16 -4.60956342e-16 -7.44072848e-17  3.26699062e-16]
 [ 6.51138304e-16  6.24136501e-16  7.87200508e-16

In [ ]:
#Predicted ratings
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt)
# Convert predicted ratings to dataframe
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = pivot_df.columns)
preds_df.head()

asin,1400501466,1400501520,1400501776,140053271X,1400698987,B00000IGBF,B00000J1TX,B00000J3II,B00000J3Q1,B00000J434,...,B00000JDK0,B00000JI2C,B00000JI2D,B00000JI38,B00000JSGF,B00000JYLO,B00001MXYW,B00001P4XA,B00001P4ZH,B00001RMCY
0,6.527667e-16,7.165676e-16,1.617621e-15,1.782956e-16,6.928216e-16,3.083530e-01,-1.117240e-01,4.627549e+00,-1.530794e-01,2.189738e-01,...,-1.117240e-01,-1.117240e-01,1.161719e-02,-3.997617e-01,5.656109e-01,1.206716e+00,4.627549e+00,4.908755e+00,4.826863e-01,-3.997617e-01
1,5.000000e+00,5.000000e+00,5.000000e+00,4.000000e+00,4.000000e+00,1.459333e-16,4.631532e-16,1.176900e-15,-6.246538e-16,4.054335e-16,...,4.660147e-16,4.660147e-16,5.196346e-16,-1.166968e-15,-2.143971e-15,-1.310975e-15,1.081945e-15,-3.449314e-16,-4.885017e-16,-1.166968e-15
2,-1.154454e-15,-9.329387e-16,-1.293519e-15,-1.330751e-15,-8.524343e-16,-9.990419e-01,3.619779e-01,1.206716e+00,4.959663e-01,-7.094596e-01,...,3.619779e-01,3.619779e-01,-3.763889e-02,1.295200e+00,2.167461e+00,1.090326e+00,1.206716e+00,2.055698e+00,4.361303e-01,1.295200e+00
3,3.361942e-16,7.577648e-16,2.089364e-15,2.096930e-15,1.211086e-15,9.249648e-02,4.966486e+00,-1.117240e-01,-4.591913e-02,4.065685e+00,...,4.966486e+00,4.966486e+00,5.003485e+00,-1.199163e-01,1.696660e-01,3.619779e-01,-1.117240e-01,2.725987e-01,1.447912e-01,-1.199163e-01
4,-1.222313e-15,-7.265770e-17,-2.526172e-15,-1.744844e-15,-5.748586e-16,3.309634e-01,-1.199163e-01,-3.997617e-01,4.835696e+00,2.350303e-01,...,-1.199163e-01,-1.199163e-01,1.246904e-02,4.570925e+00,5.607085e+00,1.295200e+00,-3.997617e-01,9.753905e-01,5.180799e-01,4.570925e+00


In [ ]:
# Recommend the items with the highest predicted ratings
def recommend_items(userID, pivot_df, preds_df, num_recommendations):
    # index starts at 0
    user_idx = userID-1
    # Get and sort the user's ratings
    sorted_user_ratings = pivot_df.iloc[user_idx].sort_values(ascending=False)
    #sorted_user_ratings
    sorted_user_predictions = preds_df.iloc[user_idx].sort_values(ascending=False)
    #sorted_user_predictions
    temp = pd.concat([sorted_user_ratings, sorted_user_predictions], axis=1)
    temp.index.name = 'Recommended Items'
    temp.columns = ['user_ratings', 'user_predictions']
    temp = temp.loc[temp.user_ratings == 0]
    temp = temp.sort_values('user_predictions', ascending=False)
    print('\nBelow are the recommended items for user(user_id = {}):\n'.format(userID))
    print(temp.head(num_recommendations))

In [ ]:
userID = 4
num_recommendations = 5
recommend_items(userID, pivot_df, preds_df, num_recommendations)


Below are the recommended items for user(user_id = 4):

                   user_ratings  user_predictions
Recommended Items                                
B00000JYLO                  0.0          0.361978
B00001P4XA                  0.0          0.272599
B00000JSGF                  0.0          0.169666
B00001P4ZH                  0.0          0.144791
B00000J4FS                  0.0          0.092496


In [ ]:
# Actual ratings given by the users
final_ratings_matrix.head()